# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mne.io.snirf import read_raw_snirf
from get_events_from_snirf import get_events_from_snirf
import itertools

In [2]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier

In [3]:
from give_channels import give_channels

In [ ]:
give_channels()

# Load Data

In [4]:
path_1 = '../data_hojjat/dec_nef/1_new_full_2/test_001_S001_c7bcdc0_5.snirf'
path_2 = "../data_hojjat/dec_nef/2/test_001_S001_7d1123e_5.snirf"
path_3 = '../data_hojjat/dec_nef/3/test_001_S001_7d1123e_5.snirf'

In [6]:
paths = [path_1, path_2, path_3]

In [ ]:
snirfs = [read_raw_snirf(path) for path in paths]

In [ ]:
datas = [snirf.get_data() for snirf in snirfs]

In [7]:
event_dfs = [get_events_from_snirf(path) for path in paths]

In [8]:
event_dfs[0]

,Timestamp,Event,Duration,Value,Experiment,Stim,Label,File,Counts,Block
0,22.644750,StartExperiment,575.932999,1.0,1.0,NaN,na,nan,0.0,False
1,22.691047,Stim,0.000000,1.0,1.0,Bee-7-1-5.Png,Bee,7_1_5.Png,3.0,True
2,23.657696,Stim,0.000000,1.0,1.0,Bee-7-1-8.Png,Bee,7_1_8.Png,2.0,False
3,24.591449,Stim,0.000000,1.0,1.0,Bee-7-1-2.Png,Bee,7_1_2.Png,1.0,False
4,25.542362,Stim,0.000000,1.0,1.0,Bee-7-1-12.Png,Bee,7_1_12.Png,0.0,False
...,...,...,...,...,...,...,...,...,...,...
596,593.856994,Stim,0.000000,1.0,1.0,GuineaPig-25-1-15.Png,GuineaPig,25_1_15.Png,0.0,False
597,594.807750,Stim,0.000000,1.0,1.0,Frog-2-1-12.Png,Frog,2_1_12.Png,3.0,True
598,595.758414,Stim,0.000000,1.0,1.0,Frog-2-1-13.Png,Frog,2_1_13.Png,2.0,False
599,596.709954,Stim,0.000000,1.0,1.0,Frog-2-1-14.Png,Frog,2_1_14.Png,1.0,False


In [ ]:
df = event_dfs[0].copy()

In [ ]:
labels = df['Label']
counts = np.zeros(np.shape(labels))
for i, label in enumerate(labels):
    cntr = 0
    j = i + 1
    while j < len(labels) and labels[j] == label:
        j += 1
        cntr += 1
    counts[i] = cntr

In [ ]:
df['Counts'] = counts

In [ ]:
counts_min = np.zeros(np.shape(counts))
for i, count in enumerate(counts):
    counts_min[i] = count -counts[i-1]

In [ ]:
counts_min[counts_min > 0]

# Some Functions

In [ ]:
def get_times(df, key):
    times = df[df.Label == key]['Timestamp'].values;
    return times

In [ ]:
def get_data(data, channels, times, index, window_size, sample_rate):
    out = np.zeros((len(times), len(channels), window_size * sample_rate))
    for time_index, time in enumerate(times):
        for channel_index, channel in enumerate(channels):
            out[time_index, channel_index, :] = data[channel, int((time+index)*sample_rate): int((time+index+window_size)*sample_rate)]
    return out

In [ ]:
def data_to_feature(data):
    [times, channels, points] = np.shape(data)
    # features = np.ones((times, channels * 3))
    _mean = np.mean(data, axis = 2)
    _min = np.min(data, axis = 2)
    _max = np.max(data, axis = 2)
    features = np.hstack((_mean, _min, _max))
    # features[:, 3:] = data[:]
    return features

In [ ]:
def make_x_y(features):
    shape = np.shape(features)
    X = np.vstack(features)
    y = np.hstack([(i+1) * np.ones(shape[1]) for i in range(len(features))])
    return X, y

In [ ]:
def classify(X, y):
    # clf = SVC(kernel='linear', C=1, random_state=42)
    # clf = MLPClassifier(random_state=1, max_iter=10000)
    clf = OneVsRestClassifier(SVC())#.fit(X, y)
    scores = cross_val_score(clf, X, y, cv=5)
    return scores

In [ ]:
def get_all_classes(events_df):
    classes = events_df.Label.unique()[1:]
    return classes

In [ ]:
def get_all_times(all_classes, events_df):
    all_times_lst = [get_times(events_df, clas) for clas in all_classes]
    all_times_array = np.array(all_times_lst, dtype=object)
    return all_times_array

In [ ]:
def get_all_data(data, channels, all_times, index, window_size, sample_rate):
    all_data_lst = [get_data(data, channels, times, index, window_size, sample_rate) for times in all_times]
    all_data_arr = np.array(all_data_lst)
    return all_data_arr

In [ ]:
def get_all_feature(all_data):
    all_feature_lst = [data_to_feature(data) for data in all_data]
    all_feature_arr = np.array(all_feature_lst)
    return all_feature_arr

In [ ]:
def select_channels(snirf, good_channels, hbo = 'HbO'):
    channel_names = [x['ch_name'] for x in snirf.info['chs']]
    out = []
    for i, name in enumerate(channel_names):
        if name[:9] in good_channels and hbo in name:
            out.append(i)
    out = np.array(out)
    return out

# Run

In [ ]:
all_classes = get_all_classes(event_dfs[0])[:]

In [ ]:
all_times = [get_all_times(all_classes, events_df) for events_df in event_dfs]

In [ ]:
datas = [np.nan_to_num(data) for data in datas]

In [ ]:
[data.shape for data in datas]

In [ ]:
good_channels = give_channels()
channels = select_channels(snirfs[0], good_channels)
# channels = [0], #, 3, 4, 6, 8, 16, 17, 18, 19, 20, 21, 22, 23]#, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
index = 0
window_size = 1
sample_rate=7
# all_data = get_all_data(data, channels, all_times, index, window_size, sample_rate)

In [ ]:
all_events = [get_all_data(data, channels, times, index, window_size, sample_rate) for data, times in zip (datas, all_times)]

In [ ]:
all_features = [get_all_feature(event) for event in all_events]

In [ ]:
[f.shape for f in all_features]

In [ ]:
all_features_arr = np.hstack(all_features)

In [ ]:
X, y = make_x_y(all_features_arr)

In [ ]:
X[:, np.max(X, 0) != 0].shape

In [ ]:
y

In [ ]:
 clf = OneVsRestClassifier(SVC()).fit(X, y)

In [ ]:
clf.predict(X[:2, :])

In [ ]:
clf.score(X, y) * 100

In [ ]:
scores = classify(X[:, np.max(X, 0) != 0], y)

In [ ]:
scores

In [ ]:
100 * (scores.mean())

In [ ]:
print('i', 'w', 's')
import warnings

warnings.simplefilter('ignore')
for index in range(10):
    all_events = [get_all_data(data, channels, times, index, window_size, sample_rate) for data, times in zip (datas, all_times)]
    all_features = [get_all_feature(event) for event in all_events]
    all_features_arr = np.hstack(all_features)
    X, y = make_x_y(all_features_arr)
    scores = classify(X[:, np.max(X, 0) != 0], y)
    print(index, window_size, 100 * (scores.mean()))

In [ ]:
X[:, np.max(X, 0) != 0]

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
tmp_df = pd.DataFrame(X)

In [ ]:
tmp_df

In [ ]:
np.sum(tmp_df.max()>0)

In [ ]:
X, y = make_x_y(all_feature)

In [ ]:
classify(X, y)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
bee_times = get_times(events_df, 'Bee')
key_times = get_times(events_df, 'Key')

In [ ]:
channel_names = [x['ch_name'] for x in snirf.info['chs']]
is_hbo = np.array([1 if channel_name.endswith(
    'HbO') else 0 for channel_name in channel_names], dtype=bool)
channel_locations_3d = [x['loc'][:3] for x in snirf.info['chs']]

In [ ]:
data = snirf.get_data()

In [ ]:
data.shape

In [ ]:
ax, fig = plt.subplots(figsize=(15, 10))
for bee in bee_times.astype(np.int):
    fig.plot(data[0, bee*7:(bee+1)*7])


In [ ]:
window_size = 1
sampling_rate = 7
channels = [0, 1]
index= 6
bee_data = get_data(data, channels, bee_times, index, window_size, sampling_rate)
key_data = get_data(data, channels, key_times, index, window_size, sampling_rate)

In [ ]:
bee_data.shape

In [ ]:
plt.plot(np.mean(bee_data, axis=0))
plt.plot(np.mean(key_data, axis=0))

In [ ]:
bee_features = data_to_feature(bee_data)
key_features = data_to_feature(key_data)

In [ ]:
bee_features.shape

In [ ]:
key_features.shape

In [ ]:
np.hstack([i * np.ones(3) for i in range(3)])

In [ ]:
clf = SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
scores.mean()

In [ ]:
clf = make_pipeline(StandardScaler(),
                    SVC(gamma='auto'))
                    # SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X, y)

In [ ]:
clf.score(X, y)

In [ ]:
len(bee_features)